# Optimizing a production system

This tutorial will guide you through the optimization functionalities of `prodsys` to optimize the configuration of a production system. With the `prodsys.optimization` package, we can utilize meta-heuristics and mathematical optimization for this task. All algorithms can be conviently used with the `prodsys.models` API. 

For this example, we will use a production system which we will load from a json-file (base_configuration.json), which can be found in the examples folder of [prodsys' github page](https://github.com/sdm4fzi/prodsys/tree/main/examples/tutorials). Download it and store it in the same folder as this notebook. Load the configuration and run a simulation with the following commands:

In [1]:
import prodsys
from prodsys.simulation import sim
sim.VERBOSE = 0

production_system = prodsys.adapters.JsonProductionSystemAdapter()	
production_system.read_data('base_configuration.json')

prodsys.adapters.add_default_queues_to_adapter(production_system)

runner = prodsys.runner.Runner(adapter=production_system)
runner.initialize_simulation()
runner.run(2880)
runner.print_results()


------------- Throughput -------------

              Output  Throughput
Product_type                    
Product_1        372    9.118083
Product_2        291    7.132694
------------- WIP -------------

Product_type
Product_1    121.834857
Product_2    137.674128
Total        258.553806
Name: WIP, dtype: float64

------------- Throughput time -------------

Product_type
Product_1    536.984894
Product_2    672.792040
Name: Throughput_time, dtype: float64

------------- Resource states -------------

                    time_increment  resource_time  percentage
Resource Time_type                                           
M1       PR             989.938681    2879.862432   34.374513
         SB            1889.923751    2879.862432   65.625487
M2       PR            2852.393003    2879.862432   99.046155
         SB               2.469429    2879.862432    0.085748
         UD              25.000000    2879.862432    0.868097
M3       PR            2850.437661    2879.862432   98.978

As already concluded in the seccond tutorial, production system configurations can be suboptimal for a certain load of products. In this example, we also see that resoures M2, M3, M4 are very heavily utilized, whereas resource M1 has only a productivy of 34.4%. In order to satify the product needs of our customers and to balance the load on the resources more evenly, we want to find a more suitable configuration with the `prodsys.optimization` package. However, for starting optimization, we also need to provide an optimization scenario, that models constraints, options, information and the objectives. Let's start by creating the constraints of the scenario with the `prodsys.models` API:

In [2]:
from prodsys.models import scenario_data

constraints = scenario_data.ScenarioConstrainsData(
    max_reconfiguration_cost=100000,
    max_num_machines=8,
    max_num_processes_per_machine=3,
    max_num_transport_resources=2
)

As you can see, the constraints consist of the maximum cost for reconfigruation and the maximumm number of machines, processes per machine and transport resources. Next, we define the options of our scenario for optimization:

In [3]:
positions = [[x*4, y*4] for x in range(4) for y in range(10)]
options = scenario_data.ScenarioOptionsData(
    transformations=[scenario_data.ReconfigurationEnum.PRODUCTION_CAPACITY],
    machine_controllers=["FIFO", "SPT"],
    transport_controllers=["FIFO", "SPT_transport"],
    routing_heuristics=["shortest_queue"],
    positions=positions
)

We specify in the scenario options the transformations that can be performed by the optmizer, which control policies and routing heuristics are available and what kind of positions are available to place resources. By choosing the transformation `PRODUCTION_CAPACITY`, the optimizer can add, remove or move production resources from the system or processes from single production resources.

At last, we need to specify our info for optimization:

In [4]:
info = scenario_data.ScenarioInfoData(
    machine_cost=40000,
    transport_resource_cost=20000,
    process_module_cost=4000,
    time_range=24*60
)

The scenario info contains information about the cost for machines, transport resources and process modules. Additionally, we specify a time range. This value is the time used for evalutation of created configurations during simulation. Since many evaluations are performed during optimization, this parameter can significantly influence the optimmization Time. For now, we specified it to one day. Lastly we can define the objectives used for optimization:

In [5]:
from prodsys.models.performance_indicators import KPIEnum
objectives = [scenario_data.Objective(
    name=KPIEnum.THROUGHPUT
)]

Currently, only reconfiguration cost, throughput time, WIP and throughput can be optimized. Yet, similar logic can also be used for optimizing the productivity. With all this data defined, we can now create our optimization scenario and additionally add it to our production system:

In [6]:
scenario = scenario_data.ScenarioData(
    constraints=constraints,
    options=options,
    info=info,
    objectives=objectives
)
production_system.scenario_data = scenario

Next, we define the hyper parameters for our optimization. At first, we will use evolutionary algorithm for our optimization, because it allows parallelization. The hyper parameters for optimization are strongly problem dependant and need to be adjusted accordingly. For this example, we will use the following parameters and run the optimization for 10 generations. Note, that this can take some time...

In [7]:
from prodsys.optimization import evolutionary_algorithm_optimization
from prodsys.optimization import evolutionary_algorithm
from prodsys.simulation import sim
sim.VERBOSE = 0

hyper_parameters = evolutionary_algorithm.EvolutionaryAlgorithmHyperparameters(
    seed=0,
    number_of_generations=10,
    population_size=16,
    mutation_rate=0.2,
    crossover_rate=0.1,
    number_of_processes=4
)
evolutionary_algorithm_optimization(
    production_system,
    hyper_parameters,
)

Best Performance:  488.0
Average Performance:  314.25
Generation 1 ________________
Best Performance:  488.0
Average Performance:  397.5625
Generation 2 ________________
Best Performance:  494.0
Average Performance:  -5820.625
Generation 3 ________________
Best Performance:  501.0
Average Performance:  489.6875
Generation 4 ________________
Best Performance:  501.0
Average Performance:  -5787.8125
Generation 5 ________________
Best Performance:  501.0
Average Performance:  -5804.0
Generation 6 ________________
Best Performance:  501.0
Average Performance:  -5782.0
Generation 7 ________________
Best Performance:  501.0
Average Performance:  499.625
Generation 8 ________________
Best Performance:  501.0
Average Performance:  498.75
Generation 9 ________________
Best Performance:  501.0
Average Performance:  499.75
Generation 10 ________________
Best Performance:  501.0
Average Performance:  499.75


All algorithms in `prodsys` can be utilized with the same interface. Also available are the following algorithms:
- `prodsys.optimization.simulated_annealing`: simulated annealing optimization for all transformations
- `prodsys.optimization.tabu_search`: tabu search for all transformations
- `prodsys.optimization.math_opt`: mathematical optimization with Gurobi, allows only optimization of the production capacity

We see in the output, that the algorithm is running and that new best solutions with a higher performance are found. We can analyze them now and see, if we can find a better configuration for our production system. Optimization core results of the objective for the individual solutions and the solutions themselves are saved as default in a `results` folder to make sure that interruptions in optimization won't delete all results. We can load them with the following command:

In [8]:
from prodsys.optimization import optimization_analysis

df = optimization_analysis.read_optimization_results_file_to_df("results/optimization_results.json", "evolutionary")
df.head()

,Generation,population_number,ID,agg_fitness,time,KPI_0,optimizer
1,0,1,1fba8cf9-1358-11ee-ad97-00155d8419b0,381.0,46.748851,381.0,evolutionary
2,0,2,1fbbc5d4-1358-11ee-9880-00155d8419b0,250.0,46.75926,250.0,evolutionary
3,0,3,1fbe36ca-1358-11ee-b5bf-00155d8419b0,248.0,46.765494,248.0,evolutionary
4,0,4,1fbe36cb-1358-11ee-a54c-00155d8419b0,37.0,46.77173,37.0,evolutionary
5,0,5,1fbf6f7a-1358-11ee-8baf-00155d8419b0,334.0,46.778027,334.0,evolutionary


`prodsys` allows us to load the optimization results as a data frame and analyze them. In this case, we just want to see the best solution found by the algorithm:

In [9]:
df.sort_values(by=["agg_fitness"], ascending=False).head()

,Generation,population_number,ID,agg_fitness,time,KPI_0,optimizer
46,3,6,334c6f32-1358-11ee-b5bf-00155d8419b0,501.0,79.573997,501.0,evolutionary
83,10,1,334c6f32-1358-11ee-b5bf-00155d8419b0,501.0,113.180321,501.0,evolutionary
56,4,7,334c6f32-1358-11ee-b5bf-00155d8419b0,501.0,81.195124,501.0,evolutionary
80,9,1,334c6f32-1358-11ee-b5bf-00155d8419b0,501.0,110.203564,501.0,evolutionary
59,5,2,334c6f32-1358-11ee-b5bf-00155d8419b0,501.0,97.801255,501.0,evolutionary


In [10]:
import os 

# Find all files i the result folder that contain the ID of the best individual
best_individual_ID = df.sort_values(by=["agg_fitness"], ascending=False).head()["ID"].values[0]
best_individual_ID = str(best_individual_ID)
files = os.listdir("results")
files = [file for file in files if best_individual_ID in file]
new_production_system = prodsys.adapters.JsonProductionSystemAdapter()
new_production_system.read_data("results/" + files[0])

In [11]:
runner = prodsys.runner.Runner(adapter=new_production_system)
runner.initialize_simulation()
runner.run(2880)

runner.print_results()


------------- Throughput -------------

              Output  Throughput
Product_type                    
Product_1        555   13.602995
Product_2        472   11.568673
------------- WIP -------------

Product_type
Product_1    35.838941
Product_2    25.536861
Total        60.614472
Name: WIP, dtype: float64

------------- Throughput time -------------

Product_type
Product_1    145.549436
Product_2    126.325789
Name: Throughput_time, dtype: float64

------------- Resource states -------------

                                                time_increment  resource_time   
Resource                             Time_type                                  
04480aa8-1358-11ee-a439-00155d8419b0 PR            2838.665276    2879.988626  \
                                     SB              41.323351    2879.988626   
04480aa9-1358-11ee-bc4f-00155d8419b0 PR            2869.915363    2879.988626   
                                     SB              10.073263    2879.988626   
04480aaa-

When comparing the results from the original production system and the new one, we see that two machines were added. However, the machines are still heavily utilized. Most likely, the optimizer did just not find a good solution, because we only ran it for 10 generations and for a small population size. Increasing these will take longer, but will more likely find better solutions. 